In [1]:
!pip install numpy==1.16.6
!pip install river

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 18.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for numpy: filename=numpy-1.16.6-cp39-cp39-linux_x86_64.whl size=3408386 sha256=7af8ebe60045991d5449e33a70db93179879d8e77b2633984ffe326c2c56d38b
  Stored in directory: /tmp/pip-ephem-wheel-cache-ywtm3c0o/wheels/ce/72/38/377a1055ac9b575ebba22e507856aecdc37f9bf6372703e5f9
Successfully built numpy
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.3
    Uninstalling numpy-1.22.3:
      Successfully uninstalled numpy-1.22.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tifffile 2022.3.16 requires numpy>=1.19.2, but you have numpy 1.16.6 which is incompatible.
tables 3.7.0 requires numpy>=1.19.0, but you have numpy 1.16.6 which is incompatible.
statsmodels 0.13.2 requires numpy>=1.17, but you have n

In [2]:
#encoding={'Benign':0, 'FTP-BruteForce':1,'DoS attacks-GoldenEye':2, 'DoS attacks-Slowloris':3,'DoS attacks-SlowHTTPTest':4, 'DoS attacks-Hulk':5,"DDOS attack-HOIC":6,"Bot":7,"DDOS attack-LOIC-UDP":8}

In [13]:
from river import stream
from river import preprocessing
from river import compose
from river import metrics
from river import stats
from river import evaluate
import pandas as pd
from river.feature_selection import PoissonInclusion
from river.imblearn import HardSamplingClassifier
from river.neighbors import KNNClassifier
from imblearn.metrics import geometric_mean_score
from river.metrics import ClassificationReport

In [14]:
data = pd.read_csv("preprocessed_multiclass_stream.csv")

In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)

In [4]:
stream_data = stream.iter_pandas(train.drop("Label", axis = 1), y = train["Label"])

In [5]:
test_data = stream.iter_pandas(test.drop("Label", axis = 1), y = test["Label"])

In [7]:
def construct_pipeline(model):
    return compose.Pipeline(
                PoissonInclusion(p=0.4), # feature selection to reduce training time
                HardSamplingClassifier( # imblearn wrapper to solve class imbalance
                            classifier=model, 
                            p=0.1,size=40)
                )

In [7]:
KNN_model = KNNClassifier()

In [12]:
model=construct_pipeline(KNN_model)

In [8]:
model = (
     preprocessing.StandardScaler() |
     KNNClassifier()
)

In [9]:
for x, y in stream_data:
    model = model.learn_one(x, y)

In [10]:
Acc_metric = metrics.Accuracy();
Rec_metric = metrics.Recall();
Pre_metric = metrics.Precision();
F1_metric = metrics.F1();

def get_metric(Acc, rec, pre, f1 ):
  Acc, rec, pre, f1 = Acc, rec, pre, f1
  return([Acc, rec, pre, f1])

In [11]:
predictions = []
true_y = []
for x, y in test_data:
    y_pred = model.predict_one(x) # make a prediction
    predictions.append(y_pred)
    true_y.append(y)
    Acc_metric = Acc_metric.update(y, y_pred)
    Rec_metric = Rec_metric.update(y, y_pred)
    Pre_metric = Pre_metric.update(y, y_pred)
    F1_metric = F1_metric.update(y, y_pred)
    model = model.learn_one(x, y)
print(get_metric(Acc_metric, Rec_metric, Pre_metric, F1_metric))  

[Accuracy: 93.03%, Recall: 69.89%, Precision: 66.90%, F1: 68.36%]


In [15]:
report = ClassificationReport()
for yt,yp in zip(true_y, predictions):
  report = report.update(yt,yp)
print(report)
print("geometric mean score :",geometric_mean_score(true_y, predictions, average='weighted'))

           Precision   Recall   F1       Support  
                                                  
       0      97.92%   90.19%   93.90%       828  
       1      66.90%   69.89%   68.36%      5171  
       2      95.83%   99.50%   97.63%      8206  
       3      96.14%   92.48%   94.28%      2075  
       4      58.29%   56.36%   57.31%      3941  
       5      98.67%   99.33%   99.00%       314  
       6      99.58%   99.98%   99.78%     16783  
       7      98.15%   99.77%   98.95%      1449  
       8     100.00%   97.33%   98.65%       337  
                                                  
   Macro      90.17%   89.43%   89.76%            
   Micro      93.03%   93.03%   93.03%            
Weighted      93.06%   93.03%   93.01%            

                 93.03% accuracy                  
geometric mean score : 0.9609445773905494
